In [ ]:
# sources:
# we used the Keras tutorial in order to understand how to create the neural network, as well as this site:
# https://becominghuman.ai/building-an-image-classifier-using-deep-learning-in-python-totally-from-a-beginners-perspective-be8dbaf22dd8
# in order to export our .h5 (keras) files into .pb (tensorflow) format, we also used another site for reference:
# https://stackoverflow.com/questions/45466020/how-to-export-keras-h5-to-tensorflow-pb

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras import backend as K

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.compat.v1.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph



classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 6, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory(
    directory="data/train",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

test_set = test_datagen.flow_from_directory(
    directory=r"data/validation",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)
classifier.fit_generator(training_set,
                         steps_per_epoch = 400,
                         epochs = 16,
                         validation_data = test_set,
                         validation_steps = 200)

classifier.save_weights('model.h5')

frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in classifier.outputs])
tf.train.write_graph(frozen_graph, "", "model.pb", as_text=False)


Found 8772 images belonging to 6 classes.
Found 2238 images belonging to 6 classes.
Epoch 1/16
400/400 [==============================] - 207s 517ms/step - loss: 0.9251 - acc: 0.4658 - val_loss: 0.8781 - val_acc: 0.5250
Epoch 2/16
400/400 [==============================] - 182s 454ms/step - loss: 0.7974 - acc: 0.5752 - val_loss: 0.8969 - val_acc: 0.5615
Epoch 3/16
400/400 [==============================] - 176s 439ms/step - loss: 0.7133 - acc: 0.6623 - val_loss: 0.7207 - val_acc: 0.6961
Epoch 4/16
400/400 [==============================] - 173s 433ms/step - loss: 0.6410 - acc: 0.7074 - val_loss: 0.7063 - val_acc: 0.7047
Epoch 5/16
400/400 [==============================] - 179s 448ms/step - loss: 0.6089 - acc: 0.7223 - val_loss: 0.7844 - val_acc: 0.6786
Epoch 6/16
400/400 [==============================] - 175s 438ms/step - loss: 0.5807 - acc: 0.7357 - val_loss: 0.6512 - val_acc: 0.7290
Epoch 7/16
400/400 [==============================] - 174s 435ms/step - loss: 0.5389 - acc: 0.7560 -